In [1]:
%matplotlib inline


TensorFlow: 静态计算图
-------------------------

实现一个隐层的全连接神经网络，优化的目标函数是预测值和真实值的欧氏距离。

这个实现使用基本的Tensorflow操作来构建一个计算图，然后多次执行这个计算图来训练网络。

Tensorflow和PyTorch最大的区别之一就是Tensorflow使用静态计算图和PyTorch使用动态计算图。

在Tensorflow里，我们首先构建计算图，然后多次执行它。




In [2]:
import tensorflow as tf
import numpy as np

# 首先构建计算图。

# N是batch大小；D_in是输入大小。
# H是隐单元个数；D_out是输出大小。
N, D_in, H, D_out = 64, 1000, 100, 10

# 输入和输出是placeholder，在用session执行graph的时候我们会feed进去一个batch的训练数据。
x = tf.placeholder(tf.float32, shape=(None, D_in))
y = tf.placeholder(tf.float32, shape=(None, D_out))

# 创建变量，并且随机初始化。 
# 在Tensorflow里，变量的生命周期是整个session，因此适合用它来保存模型的参数。
w1 = tf.Variable(tf.random_normal((D_in, H)))
w2 = tf.Variable(tf.random_normal((H, D_out)))

# Forward pass：计算模型的预测值y_pred 
# 注意和PyTorch不同，这里不会执行任何计算，而只是”定义“了计算，后面用session.run的时候才会真正的执行计算。
h = tf.matmul(x, w1)
h_relu = tf.maximum(h, tf.zeros(1))
y_pred = tf.matmul(h_relu, w2)

# 计算loss 
loss = tf.reduce_sum((y - y_pred) ** 2.0)

# 计算梯度。 
grad_w1, grad_w2 = tf.gradients(loss, [w1, w2])

# 使用梯度下降来更新参数。assign同样也只是定义更新参数的操作，不会真正的执行。
# 在Tensorflow里，更新操作是计算图的一部分；而在PyTorch里，因为是动态的”实时“的计算，
# 所以参数的更新只是普通的Tensor计算，不属于计算图的一部分。
learning_rate = 1e-6
new_w1 = w1.assign(w1 - learning_rate * grad_w1)
new_w2 = w2.assign(w2 - learning_rate * grad_w2)

# 计算图构建好了之后，我们需要创建一个session来执行计算图。
with tf.Session() as sess:
    # 首先需要用session初始化变量 
    sess.run(tf.global_variables_initializer())

    # 这是fake的训练数据
    x_value = np.random.randn(N, D_in)
    y_value = np.random.randn(N, D_out)
    for _ in range(500):
        # 用session多次的执行计算图。每次feed进去不同的数据(这里是模拟的，实际应该每次feed一个batch的数据）。
        # run的第一个参数是需要执行的计算图的节点，它依赖的节点也会自动执行，因此我们不需要手动执行forward的计算。
        # run返回这些节点执行后的值，并且返回的是numpy array
        loss_value, _, _ = sess.run([loss, new_w1, new_w2],
                                    feed_dict={x: x_value, y: y_value})
        print(loss_value)

26195068.0
20718410.0
18857554.0
17961840.0
16660671.0
14397628.0
11386147.0
8297714.0
5686458.5
3775949.0
2501313.8
1694169.4
1190028.2
872532.56
667245.25
529367.8
432513.94
361551.94
307399.1
264590.22
229887.2
201180.22
176994.42
156387.06
138673.9
123337.19
109995.77
98336.65
88120.695
79129.03
71187.24
64159.363
57915.336
52356.363
47385.73
42943.492
38966.97
35401.312
32196.355
29318.027
26724.338
24384.127
22272.96
20362.863
18632.184
17064.084
15640.318
14346.949
13170.407
12099.08
11123.058
10232.592
9419.991
8677.953
7999.657
7379.1387
6810.771
6289.5938
5811.73
5373.2812
4970.59
4600.3823
4260.0186
3946.9138
3658.5598
3393.0151
3148.2915
2922.6013
2714.3457
2522.0156
2344.3477
2180.1072
2028.2329
1887.7124
1757.7014
1637.2687
1525.637
1422.1537
1326.1802
1237.1
1154.3699
1077.5269
1006.1593
939.8011
878.1216
820.7191
767.2896
717.5639
671.26337
628.1088
587.9124
550.43115
515.50366
482.92136
452.5221
424.13965
397.6375
372.8752
349.7413
328.11423
307.8842
288.96826
271.2717